### Question 1

In [6]:
class Node(object):   
    def __init__(self, name, is_sink = False, cap=float('inf')):
        self.name = name
        self.is_sink = is_sink
        self.cap = cap

class Edge(object):
    def __init__(self, start, end, capacity, flow=0):
        self.start = start
        self.end = end
        self.capacity = capacity
        self.flow = flow
    
class Graph(object):    
    def __init__(self):
        self.nodes = {}
        self.edges = {}
    
    def get_node(self, name):
        return self.nodes[name]
    
    def add_node(self, node):
        if node.name not in self.nodes.keys():
            self.nodes[node.name] = node
            self.edges[node.name] = []
        
    def update_edge_flow(self, start, end, capacity, flow):
        for index in range(len(self.edges[start])):
            node = self.edges[start][index]
            if node[0] == end: 
                self.edges[start][index][1] = capacity
                self.edges[start][index][2] = flow
                
    def add_edge(self, edge):
        if [edge.end, edge.capacity, edge.flow] not in self.edges[edge.start]:
            self.edges[edge.start].append([edge.end, edge.capacity, 0])
            self.add_reverse_edge(edge)
    
    def add_reverse_edge(self, edge):
        self.edges[edge.end].append([edge.start, 0, 0])
        
    def get_capacity(self, start, end):
        for node in self.edges[start]:
            if node[0] == end: return node[1], node[2]
        
    def get_edges(self, node_name):
        return [node[0] for node in self.edges[node_name] if (node[1]) > 0 ] # only get edges with capacity on them
    
    def get_filled_edges(self, node_name):
        return [node[0] for node in self.edges[node_name] if (node[2]) > 0 ] # only get edges with capacity on them
    
    def __str__(self):
        return "Nodes: " + str(self.nodes.keys()) + "\n" + "Edges: " + str(self.edges)

In [7]:
class DFSSearch():
    def __init__(self, graph):
        self.graph = graph
        self.visited = set()
        self.current_path = []
        self.found = False
        self.final_path = []
        
    def set_found(self):
        self.found = False
        self.visited = set()
        self.current_path = []
        
    def get_found(self):
        return self.found
        
    def get_next(self):
        return self.next_node
    
    def find_path(self, start, end):
        self.current_path.append(start)
        if start == end and self.graph.get_node(end).cap > 0:
            self.found = True
            self.graph.get_node(end).cap -= 1
            self.final_path.append(self.current_path)
        for next_node in set(self.graph.get_edges(start)) - self.visited:
            if not self.found:
                self.visited.add(next_node)
                self.find_path(next_node, end)
                self.current_path = self.current_path[:-1]
                
    def get_final_paths(self, start, end):
        self.current_path.append(start)
        if start == end:
            self.final_path.append(self.current_path)
        for next_node in set(self.graph.get_filled_edges(start)):
            self.visited.add(next_node)
            self.get_final_paths(next_node, end)
            self.current_path = self.current_path[:-1]

In [8]:
def max_flow(dfs=None, g=None):
    flow_max = 0
    dfs.find_path('s', 't') # get a path from s to t
    while dfs.get_found():
        path = dfs.final_path[-1]
        # get min capacity along this path
        cmin = min(g.get_capacity(path[i], path[i+1])[0] for i in range(len(path)-1))
        flow_max += cmin
        # add the minimum capacity in the reverse direction of the residual graph
        for index in range(len(path)-1):
            cap, flow = g.get_capacity(path[index], path[index+1])
            rev_cap, rev_flow = g.get_capacity(path[index+1], path[index])
            g.update_edge_flow(path[index+1], path[index], flow+cmin, 0) # for reverse flow
            g.update_edge_flow(path[index], path[index+1], cap-(flow+cmin), flow+cmin) # forward flow
        dfs.set_found()
        dfs.find_path('s', 't')

In [9]:
def pop_graph(inputs):
    g = Graph()
    n = inputs[0]
    # add source and sink/target nodes
    g.add_node(Node('s'))
    g.add_node(Node('t', is_sink=True, cap=n))
    for edges in inputs[1:]:
        split_value = edges.split(' ')
        nodes = [str(i) + '-' + str(split_value[i]) for i in range(len(split_value))]
        [g.add_node(Node(node)) for node in nodes] # add all nodes to the graph
        g.add_edge(Edge(nodes[0], nodes[1], 1)) # connect the entry and exit nodes in bipartite graph
        g.add_edge(Edge("s", nodes[0], 1)) # connect the source node to all entry nodes
        g.add_edge(Edge(nodes[1], "t", 1)) # connect all exit nodes to the sink
    dfs = DFSSearch(g)
    max_flow(dfs, g)
    values = actual_paths(dfs, inputs)
    for i in values:
        print(i)

def actual_paths(dfs=None, inputs=None):
    dfs.set_found()
    dfs.final_path = []
    dfs.get_final_paths('s', 't')
    path = [str(i[1].split('-')[1] + ' ' + i[2].split('-')[1]) for i in dfs.final_path if len(i) == 4] # eliminate residual back edge paths
    return path

#### Run inputs and show outputs for part 1

In [10]:
# Input 1
inputs = [3, '1 1', '1 2', '3 2', '2 3']
pop_graph(inputs)

3 2
1 1
2 3


### Question 2

In [13]:
# class Heap:
#     def __init__(self, array = []):
#         self.array = array
      
#     def Add(self, elem):
#         self.array.append(elem)
#         current_index = len(self.array) - 1
#         while elem > self.array[self.Parent(current_index)] and current_index != 0:
#             self.array[self.Parent(current_index)], self.array[current_index] = self.array[current_index], self.array[self.Parent(current_index)] 
#             current_index = self.Parent(current_index)         
              
#     def Max(self):
#         return self.array[0]
    
#     def Remove(self):
#         self.array[0], self.array[-1] = self.array[-1], self.array[0]
#         self.array = self.array[:-1]
#         while NotHeap():
#             Drown()
    
#     def Parent(self, index):
#         return (index - 1) // 2
    
#     def Children(self, index):
#         answer = []
#         if (index + 1) * 2 - 1 < len(self.array):
#             answer.append((index + 1) * 2 - 1)
#         if (index + 1) * 2 - 1 < len(self.array):
#             answer.append((index + 1) * 2)
#         return answer
    
#     def __str__(self):
#         elems = [(elem.first, elem.second) for elem in self.array]
#         return "Elements: " + " ".join(map(str, elems))

# Define the node class
class Node:
    def __init__(self, first, second):
        self.first = first
        self.second = second
        
    def __lt__(self, other):
        return int(self.first) < int(other.first)
    
    def __gt__(self, other):
        return int(self.first) > int(other.first)
    
    def __str__(self):
        return "Node: " + str(self.first) + " " + str(self.second)

In [14]:
class BinHeap:
    def __init__(self):
        self.heapList = [0]
        self.currentSize = 0


    def percUp(self,i):
        while i // 2 > 0:
            if self.heapList[i] < self.heapList[i // 2]:
                tmp = self.heapList[i // 2]
                self.heapList[i // 2] = self.heapList[i]
                self.heapList[i] = tmp
            i = i // 2

    def insert(self,k):
        self.heapList.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def percDown(self,i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapList[i] > self.heapList[mc]:
                tmp = self.heapList[i]
                self.heapList[i] = self.heapList[mc]
                self.heapList[mc] = tmp
            i = mc

    def minChild(self,i):
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapList[i*2] < self.heapList[i*2+1]:
                return i * 2
            else:
                return i * 2 + 1

    def delMin(self):
        retval = self.heapList[1]
        self.heapList[1] = self.heapList[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapList.pop()
        self.percDown(1)
        return retval

    def buildHeap(self,alist):
        i = len(alist) // 2
        self.currentSize = len(alist)
        self.heapList = [0] + alist[:]
        while (i > 0):
            self.percDown(i)
            i = i - 1
    
    def notEmpty(self):
        return len(self.heapList) > 1

In [15]:
def growCluster(clusters, elem, count):
    found = True
    for key in clusters.keys():
        found = False
        if int(elem[0]) <= min([int(i[1]) for i in clusters[key]]) and int(elem[0]) >= min([int(i[0]) for i in clusters[key]]): 
            clusters[key].add(elem)
        else:
            for interval in clusters[key]:
                if int(elem[0]) >= int(interval[0]) and int(elem[0]) <= int(interval[1]):
                    clusters.setdefault(count, set()).add(interval)
            clusters.setdefault(count, set()).add(elem)
            break
    if found: clusters.setdefault(count, set()).add(elem)

def runHeap(inputs):
    bh = BinHeap()
    intervals = [tuple(edge.split(' ')) for edge in inputs[1:]  ]
    nodes = [Node(elem[0], elem[1]) for elem in intervals]
    bh.buildHeap(nodes)
    output_nodes = []
    while bh.notEmpty():
        output_node = bh.delMin()
        output_nodes.append((output_node.first, output_node.second))

    # form clusters
    clusters = {}
    count = 0  
    for i in output_nodes:
        growCluster(clusters, i, count)
        count += 1
    for key, values in clusters.items():
        values = sorted(values, key=lambda x: int(x[0]))
        indices = [str(intervals.index(v)+1) for v in values]
        print(' '.join(indices)) 

In [16]:
inputs = [5, '1 5', '2 6', '3 8', '7 9', '10 12']
runHeap(inputs)

1 2 3
3 4
5
